In [1]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

Using TensorFlow backend.


In [4]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes, finalAct="softmax"):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
        # CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding="same",
            input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.25))
        
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # use a *softmax* activation for single-label classification
        # and *sigmoid* activation for multi-label classification
        model.add(Dense(classes))
        model.add(Activation(finalAct))

        # return the constructed network architecture
        return model

In [6]:
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [7]:
import numpy as np
data = np.load('AMPs-ML.npz')
X = data['X']
y = data['y']
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.33, random_state=42)

INIT_LR = 1e-3
EPOCHS = 75
BS = 32
model = SmallerVGGNet.build(width=20, height=20, depth=3, classes=6, finalAct="sigmoid")

# initiallize the optimizer
opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)

# compile the model using binary cross-entropy rather than
# categorical cross-entropy -- this may seem counterintuitive for
# multi-label classification, but keep in mind that the goal here
# is to treat each output label as an independent Bernoulli distribution
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the network
print("[INFO] training network...")
model.fit(X_train, y_train,
          epochs=20,
          batch_size=64)
score = model.evaluate(X_test, y_test, batch_size=64)

[INFO] training network...
Epoch 1/20
923/923 [==============================] - 4s 4ms/step - loss: 0.9514 - acc: 0.5307
Epoch 2/20
923/923 [==============================] - 3s 3ms/step - loss: 0.8318 - acc: 0.5684
Epoch 3/20
923/923 [==============================] - 2s 3ms/step - loss: 0.7041 - acc: 0.6280
Epoch 4/20
923/923 [==============================] - 2s 3ms/step - loss: 0.6378 - acc: 0.6876
Epoch 5/20
923/923 [==============================] - 2s 3ms/step - loss: 0.5517 - acc: 0.7505
Epoch 6/20
923/923 [==============================] - 3s 3ms/step - loss: 0.4835 - acc: 0.8041A: 0s - loss: 0.4842 - acc: 0.804
Epoch 7/20
923/923 [==============================] - 2s 3ms/step - loss: 0.4296 - acc: 0.8373
Epoch 8/20
923/923 [==============================] - 2s 3ms/step - loss: 0.4215 - acc: 0.8451
Epoch 9/20
923/923 [==============================] - 2s 3ms/step - loss: 0.3948 - acc: 0.8478
Epoch 10/20
923/923 [==============================] - 2s 3ms/step - loss: 0.3764 - a

In [10]:
score[2]

IndexError: list index out of range